In [1]:
import json
import os
import cv2
import glob
import numpy as np

In [2]:
json_dir = './data/anno'
json_paths = glob.glob(os.path.join(json_dir, '*.json'))

In [3]:
label_dict = {'수각류': 0}

In [4]:
for json_path in json_paths:
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    
    image_info = json_data['images']
    annotation_info = json_data['annotations']
    print(image_info)
    print(annotation_info)
    
    file_name = image_info['filename']
    image_id = image_info['id']
    image_width = image_info['width']
    image_height = image_info['height']
    
    print(image_width,image_height)
    
    # 이미지 크기가 3024 4032 정도라 1024 768 로 리사이즈 필요
    new_width = 1024
    new_height = 768

    for ann_info in annotation_info:
        if image_id == ann_info['image_id']:
            
            img_path = os.path.join('./data/images/',file_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # 이미지 리사이징
            scale_x = new_width / img.shape[1]  # x축 비율
            scale_y = new_height / img.shape[0]  # y출 비율

            img_resized = cv2.resize(img, (new_width, new_height))

            category_name = ann_info['category_name']
            polygons = ann_info['polygon']

            # 폴리곤 좌표 생성
            points = []
            for polygon in polygons:
                x = polygon['x']
                y = polygon['y']

                # 축소된 사진에서의 포인트 좌표로 옮겨 리스트에 넣기
                resized_x = int(x*scale_x)
                resized_y = int(y*scale_y)

                points.append((resized_x,resized_y))

            cv2.polylines(img_resized, [np.array(points, np.int32).reshape((-1, 1, 2))], isClosed=True, color=(0,255,0), thickness=2)
            
            # 폴리곤 좌표를 이용하여 bbox 만들기
            x_coords = [point[0] for point in points]
            y_coords = [point[1] for point in points]
            x_min = min(x_coords)
            y_min = min(y_coords)
            x_max = max(x_coords)
            y_max = max(y_coords)

            cv2.rectangle(img_resized, (x_min, y_min), (x_max,y_max), (255,0,0), 2)

            # 좌표 yolo화
            center_x = ((x_max + x_min) / (2 * new_width))
            center_y = ((y_max + y_min) / (2 * new_height))
            yolo_w = ((x_max - x_min)) / new_width
            yolo_h = ((y_max - y_min)) / new_height

            # 라벨 이름 yolo화
            image_name_temp = file_name.replace('.jpg', '')
            label_num = label_dict[category_name]

        # yolo 파일 저장
        os.makedirs('./data/anno/yolo_anno', exist_ok=True)
        with open(f"./data/anno/yolo_anno/{image_name_temp}.txt", "a") as f:
            f.write(f"{label_num} {center_x} {center_y} {yolo_w} {yolo_h} \n") 

plt.imshow(img_resized)
plt.show()
            

{'id': '0000009076', 'filename': 'C_TP_15_00007351.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-09-03'}
[{'id': '0000012482', 'image_id': '0000009076', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 1356.312832239539, 'x': 1479.162744923189}, {'y': 1390.208149861595, 'x': 1494.5859803471228}, {'y': 1431.8069487613907, 'x': 1489.9590097199425}, {'y': 1499.5975840055025, 'x': 1486.8743626351559}, {'y': 1570.4696117607104, 'x': 1485.3320390927627}, {'y': 1632.09746198263, 'x': 1474.5357742960089}, {'y': 1667.8986534677927, 'x': 1463.354612211145}, {'y': 1702.0089519627156, 'x': 1461.3460048070976}, {'y': 1749.1614234115798, 'x': 1447.2857529787675}, {'y': 1798.3203830072043, 'x': 1446.2814492767438}, {'y': 1854.132506500518, 'x': 1447.0471266076966}, {'y': 1916.5712494885156, 'x': 1448.1835755336706}, {'y': 1971.0632433689502, 'x': 1448.1835755336706}, {'y': 2018.7437380143301, 'x': 1461.8209

{'id': '0000009091', 'filename': 'C_TP_15_00007366.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-09-03'}
[{'id': '0000005540', 'image_id': '0000009091', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 917.5291356105203, 'x': 1461.3298246161528}, {'y': 973.6104793124672, 'x': 1449.8138088329488}, {'y': 1036.8817388736381, 'x': 1455.571816724551}, {'y': 1113.0948469814123, 'x': 1482.92235420966}, {'y': 1199.3738372921, 'x': 1511.7123936676699}, {'y': 1262.645096853271, 'x': 1544.820939044381}, {'y': 1321.6024068989075, 'x': 1586.5664962584954}, {'y': 1374.807784257165, 'x': 1632.630559391311}, {'y': 1410.7573635532847, 'x': 1665.739104768022}, {'y': 1462.5247577396974, 'x': 1684.4526304157284}, {'y': 1500.6314757267517, 'x': 1688.7776670331912}, {'y': 1555.8500295255917, 'x': 1731.9627262202057}, {'y': 1604.1662640995767, 'x': 1749.2367498950116}, {'y': 1640.403440030066, 'x': 1747.50934752753

{'id': '0000009120', 'filename': 'C_TP_15_00007395.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-09-03'}
[{'id': '0000005768', 'image_id': '0000009120', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 1049.5222892792935, 'x': 2496.838664214223}, {'y': 1071.0920368569653, 'x': 2552.9792411573417}, {'y': 1079.7199358880341, 'x': 2647.9863713687737}, {'y': 1079.7199358880341, 'x': 2751.630513417609}, {'y': 1096.9757339501714, 'x': 2838.0006317916386}, {'y': 1144.42917862105, 'x': 2889.822702816056}, {'y': 1230.7081689317374, 'x': 2876.8671850599517}, {'y': 1308.3592602113565, 'x': 2846.637643629041}, {'y': 1364.4406039133034, 'x': 2794.815572604624}, {'y': 1424.8358971307848, 'x': 2725.7194779054003}, {'y': 1467.9753922861285, 'x': 2656.623383206177}, {'y': 1536.9985845346787, 'x': 2565.9347589134463}, {'y': 1575.8241301744881, 'x': 2470.927628702014}, {'y': 1631.905473876435, 'x': 2354.3279688

{'id': '0000001013', 'filename': 'C_TP_41_00001012.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-09-03'}
[{'id': '0000009879', 'image_id': '0000001013', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 1393.4056935176059, 'x': 829.8953795954579}, {'y': 1427.917289641881, 'x': 752.1622730588315}, {'y': 1492.626532374897, 'x': 747.84376714013}, {'y': 1578.9055226855844, 'x': 795.3473322458461}, {'y': 1652.242664449669, 'x': 838.5323914328608}, {'y': 1734.2077052448221, 'x': 860.1249210263682}, {'y': 1846.370392648716, 'x': 877.398944701174}, {'y': 1945.591231506007, 'x': 911.9469920507856}, {'y': 2036.184171332229, 'x': 989.680098587412}, {'y': 2075.0097169720384, 'x': 1089.0057347175457}, {'y': 2113.835262611848, 'x': 1179.6943590102765}, {'y': 2152.6608082516573, 'x': 1279.0199951404102}, {'y': 2208.742151953604, 'x': 1391.301149026648}, {'y': 2256.1955966244827, 'x': 1490.6267851567818}, {'y

{'id': '0000001018', 'filename': 'C_TP_41_00001017.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-09-03'}
[{'id': '0000009920', 'image_id': '0000001018', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 457.2786486466446, 'x': 1702.2335751731537}, {'y': 491.7902447709197, 'x': 1628.8189745552286}, {'y': 573.755285566073, 'x': 1602.90793904302}, {'y': 690.2319224855013, 'x': 1633.13748047393}, {'y': 841.2201555292048, 'x': 1693.5965633357507}, {'y': 1056.9176313059238, 'x': 1779.96668170978}, {'y': 1194.964015803024, 'x': 1827.4702468154958}, {'y': 1267.3470665328923, 'x': 1852.6888329889014}, {'y': 1355.7830316013474, 'x': 1882.9183744198115}, {'y': 1433.4341228809662, 'x': 1908.8294099320203}, {'y': 1502.4573151295165, 'x': 1934.7404454442292}, {'y': 1569.3235326202991, 'x': 1958.492227997087}, {'y': 1638.3467248688494, 'x': 1984.403263509296}, {'y': 1700.898992844098, 'x': 2012.4735519808555

...

{'id': '0000077403', 'filename': 'C_TP_53_00064001.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-10-18'}
[{'id': '0000061418', 'image_id': '0000077403', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 1827.9610840775579, 'x': 1577.6497877797117}, {'y': 1706.6488037475135, 'x': 1454.4449402555997}, {'y': 1565.7818116762824, 'x': 1313.6870544856913}, {'y': 1383.4613882477172, 'x': 1085.1555313973658}, {'y': 1208.7703163914534, 'x': 876.8287928042417}, {'y': 1172.2902300178032, 'x': 797.3355279481391}, {'y': 1408.8721021537638, 'x': 876.1362949288771}, {'y': 1579.1871569737589, 'x': 938.3847630744193}, {'y': 1804.5476748078327, 'x': 1107.2857799088774}, {'y': 2010.692149094271, 'x': 1274.2022101009982}, {'y': 2166.1405079230044, 'x': 1424.2327616543896}, {'y': 2387.231686640391, 'x': 1672.3019096308487}, {'y': 2475.7358917660563, 'x': 1774.6432591421246}, {'y': 2507.2426331560227, 'x': 1845.856

{'id': '0000077473', 'filename': 'C_TP_53_00064071.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-10-18'}
[{'id': '0000087027', 'image_id': '0000077473', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 1480.0656994880483, 'x': 1705.908080530953}, {'y': 1513.3870484754905, 'x': 1743.6186295223158}, {'y': 1577.218477264004, 'x': 1780.076036032588}, {'y': 1621.71108279611, 'x': 1806.9473456949047}, {'y': 1675.4809986743446, 'x': 1836.4906670794328}, {'y': 1762.5722635400214, 'x': 1903.7445374553238}, {'y': 1841.8215027641882, 'x': 1928.577673795889}, {'y': 1973.5977938085216, 'x': 1952.474586855594}, {'y': 1980.8480375324314, 'x': 1920.4249805088846}, {'y': 1895.9762599355245, 'x': 1817.664461223412}, {'y': 1838.1522108128522, 'x': 1718.677903701042}, {'y': 1771.5095128379687, 'x': 1643.256805718316}, {'y': 1699.8360584079453, 'x': 1564.3786651727178}, {'y': 1649.8540349267826, 'x': 1507.8128416

{'id': '0000077593', 'filename': 'C_TP_53_00064191.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-10-18'}
[{'id': '0000054973', 'image_id': '0000077593', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 2473.1577805957927, 'x': 2121.5833724816844}, {'y': 2498.488577860167, 'x': 2151.9672550847768}, {'y': 2541.0449203934068, 'x': 2207.000974672677}, {'y': 2597.8862483174603, 'x': 2272.631706463937}, {'y': 2617.8897095325556, 'x': 2337.006388155307}, {'y': 2600.756690399499, 'x': 2423.3677027246126}, {'y': 2543.3983906918033, 'x': 2460.335246902881}, {'y': 2498.9701108286486, 'x': 2488.6447151680495}, {'y': 2431.988949647348, 'x': 2526.295145536688}, {'y': 2373.953167166516, 'x': 2553.6351456567268}, {'y': 2299.978820109406, 'x': 2574.3657284347723}, {'y': 2236.603430938234, 'x': 2580.7982861028554}, {'y': 2169.2434006229923, 'x': 2585.578489435439}, {'y': 2090.6069296486767, 'x': 2595.029173819

{'id': '0000077624', 'filename': 'C_TP_53_00064222.jpg', 'width': 4032, 'height': 3024, 'device': 'camera', 'license': '', 'date_created': '2021-10-18'}
[{'id': '0000055935', 'image_id': '0000077624', 'category_id': 1, 'category_name': '수각류', 'label_type': 'IMAGE_SEGMENTATION', 'polygon': [{'y': 792.9632170694288, 'x': 2671.227525894662}, {'y': 767.4053131478214, 'x': 2638.081086761935}, {'y': 737.2097115648827, 'x': 2590.5956820040433}, {'y': 694.07313787497, 'x': 2530.1597123121805}, {'y': 659.56387892304, 'x': 2461.090032664338}, {'y': 629.3682773401013, 'x': 2396.3372079944857}, {'y': 594.8590183881711, 'x': 2327.267528346643}, {'y': 564.6634168052324, 'x': 2262.5147036767903}, {'y': 534.4678152222934, 'x': 2202.078733984928}, {'y': 515.2328779963711, 'x': 2129.3209199777284}, {'y': 499.95855627036343, 'x': 2068.2562296672327}, {'y': 491.33124153238094, 'x': 1990.5528400634096}, {'y': 487.0175841633897, 'x': 1934.4337253495373}, {'y': 478.39026942540715, 'x': 1822.195495921793}, {'

NameError: name 'plt' is not defined